<a href="https://colab.research.google.com/github/hyeonwooCH/Final_physiognomy_palmistry/blob/main/Algorithm_by_SegFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터셋 로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 1. 경로 설정
import os
import glob

test_base_path = "/content/drive/MyDrive/c. Final_Team/Split_dataset/test"
test_images_path = os.path.join(test_base_path, "images")
test_masks_path = os.path.join(test_base_path, "masks")

# SegFace 로드

In [ ]:
# 1. SegFace 저장소 클론 및 이동
!git clone https://github.com/Kartik-3004/SegFace.git
%cd SegFace

# 2. 필수 라이브러리 설치 (코랩 환경 최적화 버전)
!pip install -q timm==0.9.12 segmentation-models-pytorch albumentations python-dotenv huggingface_hub

In [ ]:
import os
from huggingface_hub import hf_hub_download

# 1. .env 경로 설정 파일 생성
root_path = "/content/SegFace"
with open(".env", "w") as f:
    f.write(f"ROOT_PATH={root_path}\n")
    f.write(f"DATA_PATH={root_path}/data\n")
    f.write(f"LOG_PATH={root_path}/logs\n")

# 2. 가중치(Weights) 다운로드
hf_hub_download(repo_id="kartiknarayan/SegFace",
                filename="convnext_celeba_512/model_299.pt",
                local_dir="./weights")

print("✅ 환경 설정 및 가중치 다운로드 완료!")

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
import os

# 1. 경로 설정 및 모듈 임포트
%cd /content/SegFace
if '/content/SegFace' not in sys.path:
    sys.path.append('/content/SegFace')

# 수정된 경로로 클래스 임포트
from network.models.segface_celeb import SegFaceCeleb

# 2. 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "/content/SegFace/weights/convnext_celeba_512/model_299.pt"
image_path = "/content/mememe_fixed.jpg" # input 바꾸기
input_res = 512

# 3. 모델 초기화 및 가중치 로드
model = SegFaceCeleb(input_res, "convnext_base")

checkpoint = torch.load(model_path, map_location=device)

# 가중치 키 추출 (에러 메시지에 기반하여 state_dict_backbone 등을 확인)
if 'state_dict_backbone' in checkpoint:
    print("state_dict_backbone 키를 발견했습니다.")
    # 저장소의 특이한 구조에 맞춰 가중치를 결합해야 할 수도 있습니다.
    # 일단 가장 확률이 높은 모델 가중치 키를 시도합니다.
    pretrained_dict = checkpoint['state_dict_backbone']
else:
    # 일반적인 경우
    pretrained_dict = checkpoint.get('model_state_dict', checkpoint)

# 현재 모델의 state_dict 가져오기
model_dict = model.state_dict()

# 가중치 이름이 'backbone.'으로 시작하지 않는 경우를 대비해 필터링 (필요 시)
# 아래는 키 이름이 매칭되지 않을 때 강제로 로드하기 위한 설정입니다.
model.load_state_dict(pretrained_dict, strict=False)

model.to(device)
model.eval()
print("모델 로드 완료!")

# 눈썹 알고리즘

In [ ]:
import torch
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "/content/SegFace/weights/convnext_celeba_512/model_299.pt"
input_res = 512

# SegFace 모델 초기화 (작성하신 방식 적용)
model = SegFaceCeleb(input_res, "convnext_base")
checkpoint = torch.load(model_path, map_location=device)
pretrained_dict = checkpoint.get('state_dict_backbone', checkpoint.get('model_state_dict', checkpoint))
model.load_state_dict(pretrained_dict, strict=False)
model.to(device)
model.eval()
print("모델 로드 및 준비 완료!")

# --- [새로운 분석 함수 정의] ---

def analyze_new_image(img_path, model, device):
    # 1. 이미지 로드 및 전처리
    img = Image.open(img_path).convert('RGB').resize((512, 512))
    img_np = np.array(img)

    # 모델 입력용 텐서 변환 (SegFace는 보통 [0, 1] 범위를 기대함)
    input_tensor = torch.from_numpy(img_np).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    input_tensor = input_tensor.to(device)

    # 2. 모델 추론
    with torch.no_grad():
        output = model(input_tensor, None, None)
        # output이 list나 tuple로 올 경우를 대비해 마지막 segmentation map 선택
        if isinstance(output, (list, tuple)):
            output = output[-1]

        # 클래스 예측 (Channel-wise Argmax)
        mask_predict = torch.argmax(output, dim=1).squeeze().cpu().numpy()

    # 3. 관상 분석 로직 (눈썹/눈 마스크 추출)
    # SegFace/CelebA 라벨 기준: 7:L-Eyebrow, 6:R-Eyebrow, 9:L-Eye, 8:R-Eye
    l_eb_mask = (mask_predict == 7).astype(np.uint8)
    r_eb_mask = (mask_predict == 6).astype(np.uint8)
    l_eye_mask = (mask_predict == 9).astype(np.uint8)

    def get_bbox_info(m):
        coords = np.column_stack(np.where(m > 0))
        if len(coords) == 0: return None
        return {'min_y': np.min(coords[:, 0]), 'min_x': np.min(coords[:, 1]),
                'max_y': np.max(coords[:, 0]), 'max_x': np.max(coords[:, 1]),
                'coords': coords}

    l_eb = get_bbox_info(l_eb_mask)
    r_eb = get_bbox_info(r_eb_mask)
    l_eye = get_bbox_info(l_eye_mask)

    analysis = {}
    if l_eb and r_eb and l_eye:
        # 미간 너비 (인당)
        analysis['glabella_width'] = r_eb['min_x'] - l_eb['max_x']
        # 눈썹 길이 비율
        eye_w = l_eye['max_x'] - l_eye['min_x']
        eb_w = l_eb['max_x'] - l_eb['min_x']
        analysis['length_ratio'] = eb_w / eye_w if eye_w > 0 else 0
        # 눈썹 기울기
        pts = l_eb['coords']
        head = pts[np.argmin(pts[:, 1])] # 가장 왼쪽
        tail = pts[np.argmax(pts[:, 1])] # 가장 오른쪽
        analysis['tilt'] = np.degrees(np.arctan2(head[0] - tail[0], tail[1] - head[1]))
        # 전택궁 거리
        analysis['palace_dist'] = l_eye['min_y'] - l_eb['max_y']
    else:
        analysis = None

    return img_np, mask_predict, analysis

# --- [실행 및 시각화] ---

test_img = "/content/h2_temp.jpg" # 실제 파일 경로
img_res, mask_res, res = analyze_new_image(test_img, model, device)

if res:
    plt.figure(figsize=(15, 7))

    # 1. 원본 이미지 + 특정 부위 오버레이
    plt.subplot(1, 2, 1)
    plt.imshow(img_res)
    # 눈(4,5)과 눈썹(2,3) 영역만 색칠해서 표시
    overlay = np.zeros_like(img_res)
    overlay[np.isin(mask_res, [6, 7])] = [255, 0, 0] # 눈썹은 빨강
    overlay[np.isin(mask_res, [8, 9])] = [0, 255, 0] # 눈은 초록
    plt.imshow(overlay, alpha=0.3)
    plt.title("Detected Features")
    plt.axis('off')

    # 2. 분석 결과 시각화
    plt.subplot(1, 2, 2)
    plt.imshow(img_res)
    info_text = (f"Glabella: {res['glabella_width']}px\n"
                 f"Length Ratio: {res['length_ratio']:.2f}\n"
                 f"Tilt: {res['tilt']:.1f}°\n"
                 f"Palace Dist: {res['palace_dist']}px")
    plt.text(10, 80, info_text, color='white', fontsize=12, fontweight='bold',
             bbox=dict(facecolor='black', alpha=0.6))
    plt.title("Physiognomy Analysis")
    plt.axis('off')
    plt.show()
else:
    print("눈썹 또는 눈 검출에 실패했습니다. 마스크를 확인하세요.")

In [ ]:
import torch
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os

# --- [모델 로드 부분은 기존과 동일하게 유지] ---
# ... (생략: model, checkpoint 로드 로직)

def analyze_new_image(img_path, model, device):
    img = Image.open(img_path).convert('RGB').resize((512, 512))
    img_np = np.array(img)

    input_tensor = torch.from_numpy(img_np).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    input_tensor = input_tensor.to(device)

    with torch.no_grad():
        output = model(input_tensor, None, None)
        if isinstance(output, (list, tuple)):
            output = output[-1]
        mask_predict = torch.argmax(output, dim=1).squeeze().cpu().numpy()

    # 1. 마스크 추출
    l_eb_mask = (mask_predict == 7).astype(np.uint8)
    r_eb_mask = (mask_predict == 6).astype(np.uint8)
    l_eye_mask = (mask_predict == 9).astype(np.uint8)

    def get_bbox_info(m):
        coords = np.column_stack(np.where(m > 0))
        if len(coords) == 0: return None
        return {'min_y': np.min(coords[:, 0]), 'min_x': np.min(coords[:, 1]),
                'max_y': np.max(coords[:, 0]), 'max_x': np.max(coords[:, 1]),
                'coords': coords}

    l_eb = get_bbox_info(l_eb_mask)
    r_eb = get_bbox_info(r_eb_mask)
    l_eye = get_bbox_info(l_eye_mask)

    analysis = {}
    if l_eb and r_eb and l_eye:
        # 2. 관상 수치 계산
        analysis['glabella_width'] = r_eb['min_x'] - l_eb['max_x']
        eye_w = l_eye['max_x'] - l_eye['min_x']
        eb_w = l_eb['max_x'] - l_eb['min_x']
        analysis['length_ratio'] = eb_w / eye_w if eye_w > 0 else 0

        pts = l_eb['coords']
        head = pts[np.argmin(pts[:, 1])]
        tail = pts[np.argmax(pts[:, 1])]
        analysis['tilt'] = np.degrees(np.arctan2(head[0] - tail[0], tail[1] - head[1]))
        analysis['palace_dist'] = l_eye['min_y'] - l_eb['max_y']

        # 3. [추가] 시각화를 위한 좌표 데이터 저장
        analysis['coords'] = {
            'l_eb_edge': l_eb['max_x'], # 왼쪽 눈썹 오른쪽 끝 x
            'r_eb_edge': r_eb['min_x'], # 오른쪽 눈썹 왼쪽 끝 x
            'eb_mid_y': (l_eb['min_y'] + l_eb['max_y']) // 2, # 눈썹 중간 높이
            'l_eb_bottom_y': l_eb['max_y'], # 눈썹 하단 y
            'l_eye_top_y': l_eye['min_y'],  # 눈 상단 y
            'l_eye_mid_x': (l_eye['min_x'] + l_eye['max_x']) // 2 # 눈 중앙 x
        }
    else:
        analysis = None

    return img_np, mask_predict, analysis

# --- [시각화 부분 수정] ---

test_img = "/content/h2_temp.jpg"
img_res, mask_res, res = analyze_new_image(test_img, model, device)

if res:
    plt.figure(figsize=(12, 8))
    plt.imshow(img_res)

    # 1. 눈/눈썹 영역 하이라이트 (투명 오버레이)
    overlay = np.zeros_like(img_res)
    overlay[np.isin(mask_res, [6, 7])] = [255, 0, 0] # Red (Eyebrows)
    overlay[np.isin(mask_res, [8, 9])] = [0, 255, 0] # Green (Eyes)
    plt.imshow(overlay, alpha=0.3)

    c = res['coords']

    # 2. 미간 너비(Glabella) 시각화 (파란색 선)
    plt.plot([c['l_eb_edge'], c['r_eb_edge']], [c['eb_mid_y'], c['eb_mid_y']],
             color='cyan', linestyle='-', linewidth=2, marker='|', markersize=10)
    plt.text((c['l_eb_edge'] + c['r_eb_edge']) // 2, c['eb_mid_y'] - 10,
             f"Glabella: {res['glabella_width']}px", color='cyan',
             fontsize=10, fontweight='bold', ha='center')

    # 3. 전택궁(Palace Dist) 시각화 (노란색 화살표)
    # 왼쪽 눈 위쪽 기준으로 표시
    plt.annotate('', xy=(c['l_eye_mid_x'], c['l_eye_top_y']),
                 xytext=(c['l_eye_mid_x'], c['l_eb_bottom_y']),
                 arrowprops=dict(arrowstyle='<->', color='yellow', lw=2))
    plt.text(c['l_eye_mid_x'] + 10, (c['l_eb_bottom_y'] + c['l_eye_top_y']) // 2,
             f"Palace: {res['palace_dist']}px", color='yellow',
             fontsize=10, fontweight='bold', va='center')

    # 4. 분석 결과 텍스트 요약박스
    info_text = (f"Glabella: {res['glabella_width']}px\n"
                 f"Palace Dist: {res['palace_dist']}px\n"
                 f"Tilt: {res['tilt']:.1f}°\n"
                 f"Length Ratio: {res['length_ratio']:.2f}")
    plt.text(20, 490, info_text, color='white', fontsize=12, fontweight='bold',
             bbox=dict(facecolor='black', alpha=0.7, edgecolor='white'))

    plt.title("Eyebrow & Eye Physiognomy Measurement")
    plt.axis('off')
    plt.show()
else:
    print("분석 실패")

In [ ]:
# 코랩 셀에 이 코드 붙이기
import os
import cv2
from PIL import Image, ImageOps
import subprocess

# 원본 이미지 경로
input_image_path = "/content/toy.jpg"

# EXIF 회전 처리된 이미지 저장할 임시 경로
fixed_image_path = "/content/toy_temp.jpg"

# 1단계: EXIF 처리
img = Image.open(input_image_path)
img = ImageOps.exif_transpose(img)  # ★ 회전 적용
img.save(fixed_image_path)

print("✅ EXIF 처리 + 추론 완료!")


# 이마 알고리즘

In [ ]:
import torch
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os

# --- [1. 모델 추론 기반 이마 분석 함수] ---

def analyze_forehead_predict(img_path, model, device):
    # 이미지 로드 및 전처리
    img = Image.open(img_path).convert('RGB').resize((512, 512))
    img_np = np.array(img)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)

    input_tensor = torch.from_numpy(img_np).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    input_tensor = input_tensor.to(device)

    # 모델 추론
    with torch.no_grad():
        output = model(input_tensor, None, None)
        if isinstance(output, (list, tuple)):
            output = output[-1]
        mask_predict = torch.argmax(output, dim=1).squeeze().cpu().numpy()

    # 라벨 정의: Skin=2, L_brow=6, R_brow=7 (SegFace/CelebA 표준)
    skin_mask = (mask_predict == 2).astype(np.uint8)
    brow_mask = np.isin(mask_predict, [6, 7]).astype(np.uint8)

    def get_coords(m):
        return np.column_stack(np.where(m > 0))

    skin_pts = get_coords(skin_mask)
    brow_pts = get_coords(brow_mask)

    analysis = {}
    if len(skin_pts) > 0 and len(brow_pts) > 0:
        # --- 영역 및 좌표 정의 ---
        face_top_y = np.min(skin_pts[:, 0])     # 헤어라인 최상단 (발제)
        face_bottom_y = np.max(skin_pts[:, 0])  # 턱 끝
        brow_top_y = np.min(brow_pts[:, 0])     # 눈썹 상단 라인
        x_start, x_end = np.min(skin_pts[:, 1]), np.max(skin_pts[:, 1])
        mid_x = (x_start + x_end) // 2

        # 1. 이마 높이 비율 (상정 비율)
        forehead_h = brow_top_y - face_top_y
        face_h = face_bottom_y - face_top_y
        analysis['forehead_ratio'] = forehead_h / face_h

        # 2. 발제선(Hairline) 형태 분석 (M자형 판별)
        hairline_pts = []
        # 이마 좌우 20%를 제외한 구간에서 최상단 피부 좌표 샘플링
        for x in range(x_start + 50, x_end - 50, 5):
            y_vals = np.where(skin_mask[:brow_top_y, x] > 0)[0]
            if len(y_vals) > 0:
                hairline_pts.append((x, y_vals[0]))

        if len(hairline_pts) > 10:
            y_only = [p[1] for p in hairline_pts]
            edge_avg = (y_only[0] + y_only[-1]) / 2
            mid_val = y_only[len(y_only)//2]
            # 중앙이 양끝보다 내려와 있으면 M자 (여기서는 임계값 15px)
            analysis['is_m_shape'] = mid_val > edge_avg + 15
            analysis['hairline_coords'] = hairline_pts
        else:
            analysis['is_m_shape'] = False
            analysis['hairline_coords'] = []

        # 3. 대칭성 및 중앙 밝기
        left_area = np.sum(skin_mask[:brow_top_y, x_start:mid_x])
        right_area = np.sum(skin_mask[:brow_top_y, mid_x:x_end])
        analysis['symmetry'] = min(left_area, right_area) / max(left_area, right_area) if max(left_area, right_area) > 0 else 0

        center_y = (face_top_y + brow_top_y) // 2
        center_roi = gray[max(0, center_y-20):min(512, center_y+20), max(0, mid_x-20):min(512, mid_x+20)]
        analysis['center_brightness'] = np.mean(center_roi) if center_roi.size > 0 else 0

        # 시각화용 추가 좌표
        analysis['viz_pts'] = {'face_top_y': face_top_y, 'brow_top_y': brow_top_y, 'mid_x': mid_x}
    else:
        analysis = None

    return img_np, mask_predict, analysis

# --- [2. 실행 및 시각화] ---

test_img = "/content/h2_temp.jpg" # 실제 파일 경로
img_res, mask_res, res = analyze_forehead_predict(test_img, model, device)

if res:
    plt.figure(figsize=(12, 8))
    plt.imshow(img_res)

    # A. 이마 영역 오버레이 (피부 중 눈썹 위 영역)
    v = res['viz_pts']
    forehead_mask = (mask_res == 1) & (np.indices((512, 512))[0] < v['brow_top_y'])
    overlay = np.zeros((*img_res.shape[:2], 4), dtype=np.uint8)
    overlay[forehead_mask] = [0, 255, 255, 80] # Cyan 투명 오버레이
    plt.imshow(overlay)

    # B. 헤어라인(Hairline) 선 그리기
    if res['hairline_coords']:
        h_pts = np.array(res['hairline_coords'])
        plt.plot(h_pts[:, 0], h_pts[:, 1], color='magenta', linewidth=2, label='Hairline')

    # C. 수직 비율 표시 (상정 높이)
    plt.annotate('', xy=(v['mid_x'], v['brow_top_y']), xytext=(v['mid_x'], v['face_top_y']),
                 arrowprops=dict(arrowstyle='<->', color='yellow', lw=2))
    plt.text(v['mid_x'] + 10, (v['face_top_y'] + v['brow_top_y']) // 2,
             f"Forehead H: {v['brow_top_y'] - v['face_top_y']}px", color='yellow', fontweight='bold')

    # D. 결과 텍스트 요약
    m_text = "M-Shape (M자형)" if res['is_m_shape'] else "Round/Straight"
    info_text = (f"[Forehead Analysis]\n"
                 f"Shape: {m_text}\n"
                 f"Ratio: {res['forehead_ratio']:.2f}\n"
                 f"Symmetry: {res['symmetry']:.2f}\n"
                 f"Brightness: {res['center_brightness']:.1f}")

    plt.text(20, 490, info_text, color='white', fontsize=12, fontweight='bold',
             bbox=dict(facecolor='black', alpha=0.7, edgecolor='white'))

    plt.title("Forehead Physiognomy Prediction")
    plt.axis('off')
    plt.show()
else:
    print("이마 영역 검출 실패")

# 눈 알고리즘

In [ ]:
import torch
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os

# --- [1. 모델 추론 기반 눈 분석 함수] ---

def analyze_eye_predict(img_path, model, device):
    # 이미지 로드 및 전처리
    img = Image.open(img_path).convert('RGB').resize((512, 512))
    img_np = np.array(img)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)

    input_tensor = torch.from_numpy(img_np).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    input_tensor = input_tensor.to(device)

    # 모델 추론
    with torch.no_grad():
        output = model(input_tensor, None, None)
        if isinstance(output, (list, tuple)):
            output = output[-1]
        mask_predict = torch.argmax(output, dim=1).squeeze().cpu().numpy()

    # 라벨 정의: 8: 왼쪽 눈(L_eye), 9: 오른쪽 눈(R_eye)
    # 여기서는 왼쪽 눈을 기준으로 상세 분석을 진행합니다.
    l_eye_mask = (mask_predict == 9).astype(np.uint8)

    def get_eye_details(eye_mask, gray_img):
        coords = np.column_stack(np.where(eye_mask > 0))
        if len(coords) < 10: return None

        ymin, xmin, ymax, xmax = np.min(coords[:, 0]), np.min(coords[:, 1]), np.max(coords[:, 0]), np.max(coords[:, 1])
        w = xmax - xmin
        h = ymax - ymin

        # 1. 기하학적 구조: 가로세로비 및 눈꼬리 각도
        l_ratio = w / h if h > 0 else 0
        head_pt = coords[np.argmin(coords[:, 1])] # 가장 왼쪽 점
        tail_pt = coords[np.argmax(coords[:, 1])] # 가장 오른쪽 점
        theta_tail = np.degrees(np.arctan2(head_pt[0] - tail_pt[0], tail_pt[1] - head_pt[1]))

        # 2. 흑백 및 안광 분석
        eye_roi = gray_img[ymin:ymax, xmin:xmax]
        # 검은자위 비중 추정 (상위 30% 어두운 영역)
        _, iris_bin = cv2.threshold(eye_roi, np.percentile(eye_roi, 30), 255, cv2.THRESH_BINARY_INV)
        r_iris = np.sum(iris_bin/255) / (w * h)

        # 안광(Specular Highlight) 추출
        _, highlight_bin = cv2.threshold(eye_roi, 240, 255, cv2.THRESH_BINARY)
        specular_strong = np.sum(highlight_bin) > 0

        # 3. 수분감 (Reflectance)
        std_val = np.std(eye_roi)
        is_watery = std_val < 15 and np.mean(eye_roi) > 100

        return {
            "l_ratio": l_ratio,
            "theta_tail": theta_tail,
            "r_iris": r_iris,
            "specular": specular_strong,
            "is_watery": is_watery,
            "bbox": [ymin, xmin, ymax, xmax],
            "points": {"head": head_pt, "tail": tail_pt}
        }

    analysis = {}
    analysis['left_eye'] = get_eye_details(l_eye_mask, gray)

    return img_np, mask_predict, analysis

# --- [2. 실행 및 상세 시각화] ---

test_img = "/content/fixed_image_temp.jpg"
img_res, mask_res, res = analyze_eye_predict(test_img, model, device)
eye = res['left_eye']

if eye:
    plt.figure(figsize=(14, 6))

    # A. 전체 얼굴 내 눈 위치 표시
    plt.subplot(1, 2, 1)
    plt.imshow(img_res)
    ymin, xmin, ymax, xmax = eye['bbox']
    # 눈 영역 박스 표시
    rect = plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, linewidth=1, edgecolor='yellow', facecolor='none')
    plt.gca().add_patch(rect)
    plt.title("Eye Detection")
    plt.axis('off')

    # B. 눈 영역 확대 및 상세 분석 시각화
    plt.subplot(1, 2, 2)
    # 주변 30px 정도 여유를 두고 확대
    crop_img = img_res[max(0, ymin-30):min(512, ymax+30), max(0, xmin-30):min(512, xmax+30)]
    plt.imshow(crop_img)

    # 텍스트 요약 up - 치켜올라감 flat - 평탄
    shape_desc = 'long' if eye['l_ratio'] > 2.5 else 'circle'
    tilt_desc = 'up' if eye['theta_tail'] > 5 else 'flat'

    info_text = (f"[Eye Analysis]\n"
                 f"Ratio: {eye['l_ratio']:.2f} ({shape_desc})\n"
                 f"Tilt: {eye['theta_tail']:.1f}° ({tilt_desc})\n"
                 f"Specular: {'Strong' if eye['specular'] else 'None'}\n"
                 f"Feature: {'Watery' if eye['is_watery'] else 'Clear'}")

    plt.text(5, 25, info_text, color='yellow', fontsize=11, fontweight='bold',
             bbox=dict(facecolor='black', alpha=0.7))

    plt.title("Detailed Eye Physiognomy")
    plt.axis('off')
    plt.show()

    # 터미널 결과 출력
    print(f"\n--- 최종 분석 리포트 ---")
    print(f"안광: {'눈빛이 형형하고 맑음' if eye['specular'] else '안광이 부족함'}")
    print(f"특징: {'눈에 수분기가 있어 도화안의 기질이 있음' if eye['is_watery'] else '단정한 눈매'}")
else:
    print("눈 영역 검출 실패")

# 점 알고리즘

In [ ]:
import os
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from PIL import Image
from torchvision import transforms

# --- [1. SegFace 추론 함수 정의] ---
def process_new_input(image_path, model, device, res=512):
    orig_img = Image.open(image_path).convert('RGB')
    img_resized = orig_img.resize((res, res))
    img_tensor = torch.from_numpy(np.array(img_resized)).permute(2, 0, 1).float().div(255).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        # SegFace 특유의 forward 인자 대응 (None 전달)
        output = model(img_tensor, labels=None, dataset=None)
        if isinstance(output, (list, tuple)):
            output = output[0]
        pred_mask = torch.argmax(output, dim=1).squeeze(0).cpu().numpy()

    return img_tensor.squeeze(0), pred_mask

# --- [2. 피부 점 탐지 함수 정의] ---
def detect_skin_moles(img_tensor, pred_mask):
    # 텐서를 Numpy로 변환
    img_np = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)

    # 2번 라벨(얼굴 피부)만 추출
    face_area = (pred_mask == 2).astype(np.uint8)

    # 전처리: 가우시안 블러로 미세 노이즈 제거
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)

    # Blackhat 연산: 밝은 배경에서 어두운 점 강조
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    blackhat = cv2.morphologyEx(blurred, cv2.MORPH_BLACKHAT, kernel)

    # 얼굴 영역으로 제한
    masked_blackhat = cv2.bitwise_and(blackhat, blackhat, mask=face_area)

    # 임계값 처리 및 컨투어 검출
    _, mole_bin = cv2.threshold(masked_blackhat, 10, 255, cv2.THRESH_BINARY)
    mole_mask = np.zeros_like(gray)
    contours, _ = cv2.findContours(mole_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 2 < area < 120: # 점 크기 필터
            perimeter = cv2.arcLength(cnt, True)
            if perimeter == 0: continue
            circularity = 4 * np.pi * (area / (perimeter * perimeter))
            if circularity > 0.6: # 원형도 필터
                cv2.drawContours(mole_mask, [cnt], -1, 255, -1)

    return img_np, mole_mask

# --- [3. 메인 실행부] ---
sample_image_path = '/content/h2_temp.jpg'

if os.path.exists(sample_image_path):
    # model과 device는 이미 선언되어 있어야 합니다 (SegFace 로드 셀 확인)
    try:
        # 1단계: SegFace 추론
        img_tensor, p_mask = process_new_input(sample_image_path, model, device)

        # 2단계: 점 탐지
        img_raw, mole_result = detect_skin_moles(img_tensor, p_mask)

        # 3단계: 시각화
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(img_raw)
        plt.title("Original Face")
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(img_raw)
        mole_cmap = mcolors.ListedColormap([(0,0,0,0), 'red'])
        plt.imshow(mole_result, cmap=mole_cmap, alpha=0.8)
        plt.title("Detected Moles (Red Dots)")
        plt.axis('off')
        plt.show()

        # 결과 분석
        num_moles = len(cv2.findContours(mole_result, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0])
        print(f"✅ 발견된 점/잡티 개수: {num_moles}개")

    except NameError as e:
        print(f"❌ 설정 에러: 모델(model)이 로드되지 않았습니다. {e}")
else:
    print(f"❌ 파일을 찾을 수 없습니다: {sample_image_path}")

# 통합

In [ ]:
import torch
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os
import matplotlib.font_manager as fm

# 폰트 설정 (이전 단계에서 설치 완료 전제)
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
nanum_font = fm.FontProperties(fname=font_path)

def analyze_face_physiognomy_full(img_path, model, device):
    # 이미지 로드 및 전처리
    img = Image.open(img_path).convert('RGB').resize((512, 512))
    img_np = np.array(img)
    gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)

    input_tensor = torch.from_numpy(img_np).permute(2, 0, 1).unsqueeze(0).float() / 255.0
    input_tensor = input_tensor.to(device)

    with torch.no_grad():
        output = model(input_tensor, None, None)
        if isinstance(output, (list, tuple)): output = output[-1]
        mask_res = torch.argmax(output, dim=1).squeeze().cpu().numpy()

    # 마스크 분리
    skin_m = (mask_res == 2).astype(np.uint8)
    l_eb_m = (mask_res == 7).astype(np.uint8)
    r_eb_m = (mask_res == 6).astype(np.uint8)
    l_eye_m = (mask_res == 9).astype(np.uint8)

    def get_info(m):
        coords = np.column_stack(np.where(m > 0))
        if len(coords) < 5: return None
        return {'min_y': np.min(coords[:, 0]), 'max_y': np.max(coords[:, 0]),
                'min_x': np.min(coords[:, 1]), 'max_x': np.max(coords[:, 1]), 'pts': coords}

    res = {}
    s_info, le_info, re_info, ly_info = get_info(skin_m), get_info(l_eb_m), get_info(r_eb_m), get_info(l_eye_m)

    # --- 1. 이마 분석 (비율 + M자형) ---
    if s_info and le_info:
        brow_top_y = min(le_info['min_y'], re_info['min_y']) if re_info else le_info['min_y']
        f_h = brow_top_y - s_info['min_y']
        res['forehead'] = {'ratio': f_h / (s_info['max_y'] - s_info['min_y']), 'h': f_h, 'top_y': s_info['min_y'], 'brow_y': brow_top_y}
        # M자형 판별
        hairline = []
        for x in range(s_info['min_x']+50, s_info['max_x']-50, 5):
            y_vals = np.where(skin_m[:brow_top_y, x] > 0)[0]
            if len(y_vals) > 0: hairline.append(y_vals[0])
        res['forehead']['is_m'] = np.mean([hairline[0], hairline[-1]]) < hairline[len(hairline)//2] - 15 if len(hairline) > 10 else False

    # --- 2. 눈썹 분석 (미간 + 기울기 + Y좌표 추가) ---
    if le_info and re_info:
        # 두 눈썹의 평균 Y축 중심값을 계산하여 시각화 기준으로 사용합니다.
        brow_mid_y = (le_info['min_y'] + le_info['max_y'] + re_info['min_y'] + re_info['max_y']) // 4

        res['brow'] = {
            'glabella': re_info['min_x'] - le_info['max_x'],
            'l_edge': le_info['max_x'],
            'r_edge': re_info['min_x'],
            'y': brow_mid_y  # <--- 이 부분이 누락되어 에러가 발생했습니다.
        }

        # 기울기 (왼쪽 눈썹 기준)
        pts = le_info['pts']
        head, tail = pts[np.argmin(pts[:, 1])], pts[np.argmax(pts[:, 1])]
        res['brow']['tilt'] = np.degrees(np.arctan2(head[0] - tail[0], tail[1] - head[1]))

    # --- 3. 눈 분석 (비율 + 전택궁 + 안광 + 수분감) ---
    if ly_info and le_info:
        eye_roi = gray[ly_info['min_y']:ly_info['max_y'], ly_info['min_x']:ly_info['max_x']]
        res['eye'] = {
            'palace': ly_info['min_y'] - le_info['max_y'],
            'ratio': (ly_info['max_x']-ly_info['min_x']) / (ly_info['max_y']-ly_info['min_y']),
            'specular': np.sum(cv2.threshold(eye_roi, 230, 255, cv2.THRESH_BINARY)[1]) > 0,
            'watery': np.std(eye_roi) < 15 and np.mean(eye_roi) > 100,
            'bbox': [ly_info['min_y'], ly_info['min_x'], ly_info['max_y'], ly_info['max_x']]
        }

    return img_np, mask_res, res

# --- 2. 시각화 및 종합 리포트 ---

def display_master_overlay_report(img, mask, res):
    plt.figure(figsize=(20, 12))

    # --- 왼쪽: 상세 측정 결과 오버레이 이미지 ---
    plt.subplot(1, 2, 1)
    plt.imshow(img)

    # 1. 부위별 세그멘테이션 오버레이
    ov = np.zeros((*img.shape[:2], 4), dtype=np.uint8)
    ov[mask == 2] = [0, 255, 255, 30] # 이마/피부
    ov[np.isin(mask, [6, 7])] = [255, 0, 0, 80] # 눈썹
    ov[np.isin(mask, [8, 9])] = [0, 255, 0, 80] # 눈
    plt.imshow(ov)

    # 2. 이마 높이 측정선 (Yellow)
    if 'forehead' in res:
        f = res['forehead']
        plt.annotate('', xy=(256, f['brow_y']), xytext=(256, f['top_y']),
                     arrowprops=dict(arrowstyle='<->', color='yellow', lw=3))
        plt.text(260, (f['top_y'] + f['brow_y']) // 2, f"Forehead: {int(f['h'])}px",
                 color='yellow', fontproperties=nanum_font, fontsize=12, fontweight='bold')

    # 3. 미간 너비 측정선 (Cyan)
    if 'brow' in res:
        b = res['brow']
        plt.plot([b['l_edge'], b['r_edge']], [b['y'], b['y']], color='cyan', lw=3, marker='|', markersize=10)
        plt.text((b['l_edge'] + b['r_edge']) // 2, b['y'] - 10, f"Glabella: {b['glabella']}px",
                 color='cyan', fontproperties=nanum_font, fontsize=12, fontweight='bold', ha='center')

        # 눈썹 기울기 가이드 (Magenta)
        # 왼쪽 눈썹 끝부분에 각도 표시
        plt.text(b['l_edge'] - 50, b['y'] - 30, f"Tilt: {res['brow']['tilt']:.1f}°",
                 color='magenta', fontproperties=nanum_font, fontsize=11, fontweight='bold')

    # 4. 전택궁 측정선 (White)
    if 'eye' in res:
        e = res['eye']
        # 왼쪽 눈 중앙 기준 수직선
        eye_x_mid = (e['bbox'][1] + e['bbox'][3]) // 2
        plt.annotate('', xy=(eye_x_mid, e['bbox'][0]), xytext=(eye_x_mid, res['brow']['y']),
                     arrowprops=dict(arrowstyle='->', color='white', lw=2))
        plt.text(eye_x_mid + 5, (e['bbox'][0] + res['brow']['y']) // 2, f"Palace: {e['palace']}px",
                 color='white', fontproperties=nanum_font, fontsize=11, fontweight='bold')

    plt.title("◈ 상세 측정 지표 오버레이 ◈", fontproperties=nanum_font, fontsize=18)
    plt.axis('off')

    # --- 오른쪽: 종합 분석 텍스트 리포트 ---
    plt.subplot(1, 2, 2)
    plt.axis('off')
    y_pos = 0.95
    plt.text(0, y_pos, "◈ 종합 관상 분석 리포트 ◈", fontproperties=nanum_font, fontsize=22, fontweight='bold', color='navy')

    if 'forehead' in res:
        f = res['forehead']
        y_pos -= 0.12
        m_txt = "M자형 (예술적 감각)" if f['is_m'] else "라운드/직선형 (안정적)"
        plt.text(0, y_pos, f"■ 이마 (상정): 비율 {f['ratio']:.2f} | 형태: {m_txt}\n   해석: {'이마가 넓어 관운과 총명이 따름' if f['ratio'] > 0.3 else '균형 잡힌 지성미'}",
                 fontproperties=nanum_font, fontsize=14)

    if 'brow' in res:
        b = res['brow']
        y_pos -= 0.15
        g_txt = "넓은 미간 (개방적)" if b['glabella'] > 45 else "좁은 미간 (섬세함)"
        plt.text(0, y_pos, f"■ 눈썹 (인당): 미간 {b['glabella']}px | 기울기 {b['tilt']:.1f}°\n   해석: {g_txt}, {'눈썹이 올라가 강직한 성품' if b['tilt'] > 10 else '차분한 성품'}",
                 fontproperties=nanum_font, fontsize=14)

    if 'eye' in res:
        e = res['eye']
        y_pos -= 0.18
        s_txt = "유 (총명함)" if e['specular'] else "무"
        w_txt = "도화안 기질 (매력적)" if e['watery'] else "맑고 담백함"
        plt.text(0, y_pos, f"■ 눈 (전택궁): 거리 {e['palace']}px | 비율 {e['ratio']:.2f}\n   안광: {s_txt} | 특이사항: {w_txt}\n   해석: {'전택궁이 넓어 주거 운이 좋음' if e['palace'] > 20 else '자수성가형 눈매'}",
                 fontproperties=nanum_font, fontsize=14)

    plt.tight_layout()
    plt.show()

# 실행
img_res, mask_res, final_res = analyze_face_physiognomy_full("/content/test9.jpg", model, device)
display_master_overlay_report(img_res, mask_res, final_res)
